# KeyFrame Sinopsis

If you have any question ask Pedro Martínez-Huertas:
pedromarhuer03@gmail.com

## Libraries

In [1]:
import cv2
import os
from tqdm import tqdm
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from utils.grouping import grouping
from utils.inference import yolo_prediction
from utils.functions import clean_noise
from utils.sinopsis import sinopsis, time_series
from utils.video import video_to_frames, frames_to_video

## Convert Video to Frames

In [ ]:
video_path = "Video.mp4"

#------------------------------------------------------------------------
output_folder = "Frames_video"

#------------------------------------------------------------------------
video_to_frames(video_path, output_folder, prefix='frame', skip=1)

## Clustering module

In [ ]:
folder = 'Frames_video'

#-----------------------------------------------------------------------------------------------
# Technique used to construct the distance matrix
# 1)rbf 2)cosine
similarity = 'rbf'

#-----------------------------------------------------------------------------------------------
# Technique used for dimensional reduction
# 1)MDS 2)FastMDS
reducction = 'FastMDS'

#-----------------------------------------------------------------------------------------------
# Technique used for clustering
# 1)KMEANS 2)HDBSCAN
clustering = 'HDBSCAN'

#-----------------------------------------------------------------------------------------------
# Function
# Parameters
# n_clusters=2
#    plot_similitud=False------- to plot similarity matrix
#    plot_reduction=False------- to plot reduction map
#    plot_clustering=False------ to plot clustering map
#    plot_images=False---------- to plot representative images from clusters
df = grouping(folder,similarity,reducction,clustering,
              n_clusters=2,plot_similitud=False,plot_reduction=False,plot_clustering=True,plot_images=True)

In [ ]:
#Optional: Clean noise (unlabeled images)
df = clean_noise(df)

## Prediction module

In [ ]:
#---------------------------------------------------------------------------------------------
model_path = "model.pt"

#----------------------------------------------------------------------------------------------
# Images folder
input = "Frames_video"

#-----------------------------------------------------------------------------------------------
#Prediction folder
output = "Frames_video_predicted"

#-----------------------------------------------------------------------------------------------
# Filter for the images to be predicted
# None: all classes
filter = [1,3,4,8]
#Threshold for the confidence of the prediction (0-1)
threshold = 0.2

#-----------------------------------------------------------------------------------------------
#Creates the output folder and a DataFrame with the predictions and frame evaluation
df_inferenced = yolo_prediction(df,model_path,input,output,filter,conf_threshold=threshold)

## Fusion module

In [ ]:
yolo_folder = "Frames_video_predicted"

output_folder = "Frames_video_sinopsis"

#-----------------------------------------------------------------------------------------------
# Return a list of frames and makes a folder with the sinopsis frames
#Parameters
#    plot_series=False-------- to plot the time series
#    plot_frames=True--------- to plot the sinopsis frames with the predictions

frames = sinopsis(df_inferenced, yolo_folder,output_folder, plot_series=True, plot_frames=True)

## Sinopsis Video

In [ ]:
# Sinopsis frames folder
frames_folder = "Frames_video_sinopsis"    

#-----------------------------------------------------------------------------------------------
# Video path
output_video   = "Video_Sinopsis.mp4"                

#-----------------------------------------------------------------------------------------------
# Frames per second for the video (Real time:30fps)
fps= 30                                         

#-----------------------------------------------------------------------------------------------
frames_to_video(frames_folder, output_video, fps=fps, prefix="frame")